In [20]:
import sys
sys.path.append("../")


import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from openpyxl import load_workbook

from linearmodels import PanelOLS

from library import print_statistics
from library import analysis
from library import start

start.data_path

'/Users/kylie/dofis/data/'

In [27]:
data = pd.read_csv(os.path.join(start.data_path, 'clean', 'master_data_district.csv'),
                  sep=",", low_memory= False)
data = data[data.doi == True]
print(data[data.doi == True].district.nunique())
print(data.district.nunique())

822
822


In [36]:
#convert year to datetime
df = data.reset_index()
df['year'] = pd.to_datetime(df['year'], format='%Y')
df = df.set_index(['year', 'district'])
df.index = df.index.swaplevel(0,1)
df[['doi_year','treatpost','treatpost', 'pre5', 'pre4', 'pre3', 'pre2', 'pre1', 'post1', 'post2', 'post3']].sample(5, random_state = 10)

,,doi_year,treatpost,treatpost,pre5,pre4,pre3,pre2,pre1,post1,post2,post3
district,year,,,,,,,,,,,
109911,2016-01-01,2018.0,False,False,0,0,0,1,0,0,0,0
19902,2015-01-01,2018.0,False,False,0,0,1,0,0,0,0,0
84906,2012-01-01,2017.0,False,False,1,0,0,0,0,0,0,0
153905,2016-01-01,2018.0,False,False,0,0,0,1,0,0,0,0
167901,2012-01-01,2017.0,False,False,1,0,0,0,0,0,0,0


In [7]:
subjects = ['m_3rd_avescore', 'm_4th_avescore',  'm_5th_avescore', 
            'm_6th_avescore', 'm_7th_avescore', 'm_8th_avescore',
            'alg_avescore',
            'r_3rd_avescore', 'r_4th_avescore', 'r_5th_avescore',
            'r_6th_avescore', 'r_7th_avescore', 'r_8th_avescore',
            'eng1_avescore',  'bio_avescore']
math_tests = ['m_3rd_avescore', 'm_4th_avescore', 'm_5th_avescore',
              'm_6th_avescore', 'm_7th_avescore', 'm_8th_avescore', 'alg_avescore']
reading_tests = ['r_3rd_avescore', 'r_4th_avescore', 'r_5th_avescore',
              'r_6th_avescore', 'r_7th_avescore', 'r_8th_avescore', 'eng1_avescore']

In [8]:
#convert year to datetime
df = data.reset_index()
df['year'] = pd.to_datetime(df['year'], format='%Y')
#add column year to index
df = df.set_index(['year', 'campus'])
#swap indexes
df.index = df.index.swaplevel(0,1)
df[['district', 'doi_year','treatpost']].sample(5, random_state = 8)

,,district,doi_year,treatpost
campus,year,,,
101914042,2019-01-01,101914,2018.0,True
57905049,2016-01-01,57905,2018.0,False
243903041,2017-01-01,243903,2018.0,False
170902118,2018-01-01,170902,2017.0,True
227901112,2016-01-01,227901,2017.0,False


In [56]:
df[df.type_suburban == 1]

,,index,Unnamed: 0,distname,distischarter,rating_academic,rating_financial,type,type_description,cntyname,students_amind_num,...,pre4,pre3,pre2,pre1,post1,post2,post3,max_school_avescore,min_school_avescore,school_spread
district,year,,,,,,,,,,,,,,,,,,,,,
2901,2012-01-01,7,7,ANDREWS ISD,N,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,ANDREWS,6,...,0,0,0,0,0,0,0,-0.098812,-0.803456,0.704644
5902,2012-01-01,17,17,HOLLIDAY ISD,N,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,ARCHER,7,...,0,0,0,0,0,0,0,0.654317,-0.489598,1.143915
8903,2012-01-01,27,27,BRAZOS ISD,N,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,AUSTIN,0,...,0,0,0,0,0,0,0,0.449966,-0.753542,1.203509
14903,2012-01-01,47,47,BELTON ISD,N,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,BELL,86,...,0,0,0,0,0,0,0,1.039160,-0.822535,1.861696
14909,2012-01-01,52,52,TEMPLE ISD,N,NaN,NaN,D,OTHER CENTRAL CITY SUBURBAN,BELL,51,...,0,0,0,0,0,0,0,0.559460,-1.257151,1.816611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246908,2018-01-01,8478,8478,LIBERTY HILL ISD,N,A,Pass,D,OTHER CENTRAL CITY SUBURBAN,WILLIAMSON,14,...,0,0,0,0,0,1,0,1.580671,0.232017,1.348654
246909,2018-01-01,8479,8479,ROUND ROCK ISD,N,A,Pass,B,MAJOR SUBURBAN,WILLIAMSON,206,...,0,0,0,0,0,1,0,3.593623,-1.462001,5.055624
246911,2018-01-01,8480,8480,TAYLOR ISD,N,C,Pass,D,OTHER CENTRAL CITY SUBURBAN,WILLIAMSON,3,...,0,0,0,0,0,1,0,0.106133,-0.384609,0.490743


# Specifications

In [41]:
gdid_model = 'avescores ~ + 1 + treatpost + EntityEffects + TimeEffects'
linear_gdid_model = 'avescores ~ + 1 + treatpost + yearpost + yearpre + C(yera) + C(district)'
event_study_model = 'avescores ~ + 1 + pre5 + pre4 + pre3 + pre2 + post1 + post2 + post3 + C(year) + C(district)'

In [49]:
gdid_model = 'school_spread ~ + 1 + treatpost + EntityEffects + TimeEffects'
linear_gdid_model = 'school_spread ~ + 1 + treatpost + yearpost + yearpre + EntityEffects + TimeEffects'
event_study_model = 'school_spread ~ + 1 + pre5 + pre4 + pre3 + pre2 + post1 + post2 + post3 + + EntityEffects + TimeEffects'

In [58]:
mod = PanelOLS.from_formula(event_study_model, df)
res = mod.fit(cov_type='clustered', cluster_entity = True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:          school_spread   R-squared:                        0.0007
Estimator:                   PanelOLS   R-squared (Between):              0.0023
No. Observations:                5746   R-squared (Within):              -0.0037
Date:                Tue, Apr 07 2020   R-squared (Overall):              0.0017
Time:                        13:43:54   Log-likelihood                   -1784.7
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.4772
Entities:                         822   P-value                           0.8518
Avg Obs:                       6.9903   Distribution:                  F(7,4911)
Min Obs:                       3.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             0.7109
                            

In [59]:
mod = PanelOLS.from_formula(gdid_model, df)
res = mod.fit(cov_type='clustered', cluster_entity = True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:          school_spread   R-squared:                     1.933e-06
Estimator:                   PanelOLS   R-squared (Between):          -6.617e-05
No. Observations:                5746   R-squared (Within):           -9.743e-05
Date:                Tue, Apr 07 2020   R-squared (Overall):          -6.828e-05
Time:                        13:51:02   Log-likelihood                   -1786.6
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.0095
Entities:                         822   P-value                           0.9223
Avg Obs:                       6.9903   Distribution:                  F(1,4917)
Min Obs:                       3.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             0.0065
                            

In [60]:
data[data.year == 2019]

,Unnamed: 0,year,district,distname,distischarter,rating_academic,rating_financial,type,type_description,cntyname,...,pre4,pre3,pre2,pre1,post1,post2,post3,max_school_avescore,min_school_avescore,school_spread
